## Setup and Imports

In [ ]:
!pip install langchain-google-genai google-generativeai langchain langchain-community

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install google-search-results

In [ ]:
!pip install langchain_classic langchain-community

In [ ]:
!pip install wikipedia

In [ ]:
!pip install -U ddgs

In [ ]:
# langgraph_rag_gemini.py
import os
from pathlib import Path
from typing import List, Any
from pydantic import BaseModel, Field

# LangGraph
from langgraph.graph import StateGraph, END

# LLM (Gemini)
from langchain_google_genai import ChatGoogleGenerativeAI

# Text splitting / embeddings / FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Retrieval chain
from langchain_classic.chains import RetrievalQA

In [ ]:
from typing import Dict, List, Optional
from langchain_classic.tools import Tool

In [ ]:
from langchain_classic.schema import SystemMessage
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_classic.agents import initialize_agent, AgentType, load_tools, AgentExecutor
from langchain.chat_models import init_chat_model

In [ ]:
from google.colab import userdata
google_api_key=userdata.get('GAPI')

## Agent Roles and Multi-Agent System Definition

In [ ]:

# Define agent roles and their system messages
AGENT_ROLES = {
    "coordinator": SystemMessage(content="""You are a coordinator agent. Your role is to:
        1. Break down complex tasks into subtasks
        2. Delegate tasks to appropriate specialist agents
        3. Synthesize results from different agents
        4. Ensure the final output is coherent and complete"""),

    "researcher": SystemMessage(content="""You are a research agent. Your role is to:
        1. Gather information from multiple sources
        2. Verify facts and cross-reference information
        3. Identify key points and relevant details
        4. Provide well-sourced information"""),

    "analyst": SystemMessage(content="""You are an analyst agent. Your role is to:
        1. Analyze gathered information
        2. Identify patterns and insights
        3. Compare and contrast different viewpoints
        4. Draw meaningful conclusions"""),

    "writer": SystemMessage(content="""You are a writer agent. Your role is to:
        1. Create clear and engaging content
        2. Structure information logically
        3. Maintain consistent style and tone
        4. Ensure accuracy while being accessible""")
}


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
class MultiAgentSystem:
    def __init__(self):
        """Initialize the multi-agent system with specialized agents."""
        # Initialize base LLM
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0,google_api_key=google_api_key)

        # Initialize tools
        self.tools = self._initialize_tools()

        # Initialize specialized agents
        self.agents = self._initialize_agents()

    def _initialize_tools(self) -> Dict[str, List[Tool]]:
        """Initialize different tools for different agents."""
        # Research tools
        wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        search = DuckDuckGoSearchRun()

        # Create a basic note-taking tool that all agents can use
        def make_note(text: str) -> str:
            """Tool for making notes or storing intermediate results."""
            return f"Noted: {text}"

        # Basic tools that all agents should have
        basic_tools = [
            Tool(
                name="NoteTaking",
                func=make_note,
                description="Use this to make notes or store intermediate results"
            )
        ]

        # Specialized research tools
        research_tools = [
            Tool(name="Wikipedia", func=wiki.run,
                 description="Search Wikipedia for factual information"),
            Tool(name="Web Search", func=search.run,
                 description="Search the internet for recent or specific information")
        ]

        return {
            "coordinator": basic_tools,  # Coordinator gets basic tools
            "researcher": research_tools + basic_tools,  # Researcher gets all tools
            "analyst": [Tool(name="Wikipedia", func=wiki.run,
                           description="Verify facts and gather additional context")] + basic_tools,
            "writer": basic_tools  # Writer gets basic tools
        }

    def _initialize_agents(self) -> Dict[str, AgentExecutor]:
        """Initialize different specialized agents."""
        agents = {}

        for role, system_message in AGENT_ROLES.items():
            agents[role] = initialize_agent(
                tools=self.tools[role],
                llm=self.llm,
                agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                verbose=True,
                handle_parsing_errors=True,
                system_message=system_message
            )

        return agents

    def process_task(self, task: str) -> str:
        """Process a task using the multi-agent system."""
        print(f"\n🎯 Processing task: {task}\n")

        # 1. Coordinator breaks down the task
        print("\n🎭 Coordinator Agent: Breaking down the task...")
        subtasks = self.agents["coordinator"].run(
            f"Break this task into subtasks for research, analysis, and writing: {task}"
        )

        # 2. Researcher gathers information
        print("\n🔍 Research Agent: Gathering information...")
        research_results = self.agents["researcher"].run(
            f"Research the following aspects: {subtasks}"
        )

        # 3. Analyst processes the information
        print("\n🧮 Analysis Agent: Analyzing information...")
        analysis = self.agents["analyst"].run(
            f"Analyze this information and identify key insights: {research_results}"
        )

        # 4. Writer creates the final response
        print("\n✍️ Writing Agent: Creating final response...")
        final_response = self.agents["writer"].run(
            f"Create a well-structured response using this analysis: {analysis}"
        )

        return final_response


In [ ]:
# Example usage
if __name__ == "__main__":
    # Initialize the multi-agent system
    mas = MultiAgentSystem()

    # Example task
    task = "Explain the evolution of AI coding assistants, their current capabilities, and future potential. Include specific examples and comparisons."

    try:
        # Process the task
        result = mas.process_task(task)
        print("\n🎉 Final Result:\n", result)
    except Exception as e:
        print(f"\n❌ Error: {e}")
